In [4]:
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model

# path to the model weights files.
weights_path = '/home/mksnkv/models/pretrained/vgg16_weights.h5'
top_model_weights_path = '/home/mksnkv/models/top_tuned/bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '/home/mksnkv/Documents/classification/class/data/train'
validation_data_dir = '/home/mksnkv/Documents/classification/class/data/validation'
nb_train_samples = 6500
nb_validation_samples = 2500
epochs = 50
batch_size = 20

# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False)
print('VGG16 model loaded.')

VGG16 model loaded.


In [5]:
# build a classifier model to put on top of the convolutional model
# top_model = Sequential()
# top_model.add(Flatten(input_shape=model.output_shape[1:]))
# top_model.add(Dense(256, activation='relu'))
# top_model.add(Dropout(0.5))
# top_model.add(Dense(None, 5, activation='softmax'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model = load_model(top_model_weights_path)

# add the model on top of the convolutional base
model = Model(inputs= base_model.input, outputs= top_model(base_model.output))

# set the first 15 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

In [6]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save('/home/mksnkv/models/fine_tuned/fine_tuned_model.h5')

Found 6500 images belonging to 5 classes.
Found 2500 images belonging to 5 classes.
Epoch 1/50
325/325 [==============================] - 1414s 4s/step - loss: 9.4695 - accuracy: 0.4037 - val_loss: 12.4625 - val_accuracy: 0.2268
Epoch 2/50
325/325 [==============================] - 1399s 4s/step - loss: 12.8697 - accuracy: 0.2015 - val_loss: 12.4625 - val_accuracy: 0.2268
Epoch 3/50
325/325 [==============================] - 1398s 4s/step - loss: 12.8697 - accuracy: 0.2015 - val_loss: 12.4625 - val_accuracy: 0.2268
Epoch 4/50
325/325 [==============================] - 1398s 4s/step - loss: 12.8697 - accuracy: 0.2015 - val_loss: 12.4625 - val_accuracy: 0.2268
Epoch 5/50
325/325 [==============================] - 1397s 4s/step - loss: 12.8697 - accuracy: 0.2015 - val_loss: 12.4625 - val_accuracy: 0.2268
Epoch 6/50
236/325 [====================>.........] - ETA: 5:42 - loss: 12.8228 - accuracy: 0.2044

KeyboardInterrupt: 